In [1]:
import MetaTrader5 as mt5
import time

def enviar_ordem(symbol, volume, tipo_ordem, comentario="Ordem via Python", numero_magico=999):
    print(f"Tentando enviar ordem: {comentario} | Ativo: {symbol} | Volume: {volume}")

    # 1. Inicializa conexão
    if not mt5.initialize():
        print("Erro ao iniciar conexão com MetaTrader 5:", mt5.last_error())
        return False

    try:
        # 2. Verifica se a negociação está permitida
        terminal = mt5.terminal_info()
        if terminal is None:
            print("ERRO: Não foi possível obter informações do terminal.")
            return False

        if not terminal.trade_allowed:
            print("ERRO: A negociação automatizada está desativada no MetaTrader 5.")
            print("Ative o botão 'AutoTrading' e tente novamente.")
            return False

        # 3. Verifica se o ativo existe e está visível
        symbol_info = mt5.symbol_info(symbol)
        if symbol_info is None:
            print(f"O ativo '{symbol}' não foi encontrado.")
            return False

        if not symbol_info.visible:
            if not mt5.symbol_select(symbol, True):
                print(f"Erro ao adicionar '{symbol}' à Observação de Mercado.")
                return False
            time.sleep(1)

        # 4. Pega o preço de mercado
        tick = mt5.symbol_info_tick(symbol)
        if tick is None:
            print(f"Erro ao obter tick para o ativo '{symbol}'.")
            return False

        price = tick.ask if tipo_ordem == mt5.ORDER_TYPE_BUY else tick.bid
        if price == 0:
            print(f"Preço inválido para o ativo '{symbol}'. Verifique a liquidez ou se o mercado está aberto.")
            return False

        # 5. Monta o request
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(volume),
            "type": tipo_ordem,
            "price": price,
            "deviation": 20,
            "magic": numero_magico,
            "comment": comentario,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,  # IOC é mais aceito que FOK
        }

        # 6. Envia a ordem
        result = mt5.order_send(request)

        # 7. Trata o resultado
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("--- FALHA AO ENVIAR ORDEM ---")
            print("Retcode:", result.retcode)
            print("Mensagem:", result.comment)
            print("Detalhes:", result)
            return False

        print("--- ORDEM ENVIADA COM SUCESSO ---")
        print(f"Ticket: {result.order} | Preço: {result.price} | Volume: {result.volume}")
        return True

    finally:
        mt5.shutdown()


In [14]:
import MetaTrader5 as mt5

if not mt5.initialize():
    print("Erro ao iniciar MT5:", mt5.last_error())
else:
    info = mt5.terminal_info()
    print("Conectado com sucesso!")
    print(info)
    mt5.shutdown()


Conectado com sucesso!
TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=5120, maxbars=100000, codepage=1252, ping_last=122831, community_balance=0.0, retransmission=0.3536141445657826, company='MetaQuotes Ltd.', name='MetaTrader 5', language='Portuguese (Brazil)', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\dimit\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\dimit\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')


In [17]:
# --- Parâmetros Editáveis ---
ativo_acao = "PETR4"
volume_compra = 100.0
# ----------------------------
enviar_ordem(
    symbol=ativo_acao,
    volume=volume_compra,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="1. Compra de Acao"
)

Tentando enviar ordem: 1. Compra de Acao | Ativo: PETR4 | Volume: 100.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132755999 | Preço: 31.5 | Volume: 100.0


True

In [19]:
# --- Parâmetros Editáveis ---
ativo_acao = "VALE3"
volume_venda = 100
# ----------------------------
enviar_ordem(
    symbol=ativo_acao,
    volume=volume_venda,
    tipo_ordem=mt5.ORDER_TYPE_SELL,
    comentario="2. Venda de Acao"
)

Tentando enviar ordem: 2. Venda de Acao | Ativo: VALE3 | Volume: 100
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132769528 | Preço: 51.99 | Volume: 100.0


True

In [8]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O ticker da opção muda constantemente! Verifique o ticker vigente no seu MT5.
# Exemplo para JULHO/25: PETRG400 (PETR4, G=Julho, Call, strike R$40.00)
ticker_call = "PETR3F"
quantidade_opcoes = 2
# ----------------------------
enviar_ordem(
    symbol=ticker_call,
    volume=quantidade_opcoes,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="3. Compra de Call"
)

Tentando enviar ordem: 3. Compra de Call | Ativo: PETR3F | Volume: 2
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2134547996 | Preço: 33.89 | Volume: 2.0


True

In [7]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O ticker da opção muda constantemente! Verifique o ticker vigente no seu MT5.
ticker_call = "PETR3F"
quantidade_opcoes = 1
# ----------------------------
enviar_ordem(
    symbol=ticker_call,
    volume=quantidade_opcoes,
    tipo_ordem=mt5.ORDER_TYPE_SELL,
    comentario="4. Venda de Call"
)

Tentando enviar ordem: 4. Venda de Call | Ativo: PETR3F | Volume: 1
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2134534968 | Preço: 33.89 | Volume: 1.0


True

In [16]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O ticker da opção muda constantemente! Verifique o ticker vigente no seu MT5.
# Exemplo para JULHO/25: VALES600 (VALE3, S=Julho, Put, strike R$60.00)
ticker_put = "USIMS65"
quantidade_opcoes = 150
# ----------------------------
enviar_ordem(
    symbol=ticker_put,
    volume=quantidade_opcoes,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="5. Compra de Put"
)

Tentando enviar ordem: 5. Compra de Put | Ativo: USIMS65 | Volume: 150
Preço inválido para o ativo 'USIMS65'. Verifique a liquidez ou se o mercado está aberto.


False

In [17]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O ticker da opção muda constantemente! Verifique o ticker vigente no seu MT5.
ticker_put = "USIMS65"
quantidade_opcoes = 1
# ----------------------------
enviar_ordem(
    symbol=ticker_put,
    volume=quantidade_opcoes,
    tipo_ordem=mt5.ORDER_TYPE_SELL,
    comentario="6. Venda de Put"
)

Tentando enviar ordem: 6. Venda de Put | Ativo: USIMS65 | Volume: 1
Preço inválido para o ativo 'USIMS65'. Verifique a liquidez ou se o mercado está aberto.


False

In [24]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O contrato futuro vence! O contrato vigente para Mini-Índice é WINQ25 (Agosto/25).
contrato_ibov = "WINQ25"
numero_contratos = 1.0
# ----------------------------
enviar_ordem(
    symbol=contrato_ibov,
    volume=numero_contratos,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="7. Compra Futuro Ibov"
)

Tentando enviar ordem: 7. Compra Futuro Ibov | Ativo: WINQ25 | Volume: 1.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132780839 | Preço: 139425.0 | Volume: 1.0


True

In [25]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O contrato futuro vence! O contrato vigente para Mini-Índice é WINQ25 (Agosto/25).
contrato_ibov = "WINQ25"
numero_contratos = 1.0
# ----------------------------
enviar_ordem(
    symbol=contrato_ibov,
    volume=numero_contratos,
    tipo_ordem=mt5.ORDER_TYPE_SELL,
    comentario="8. Venda Futuro Ibov"
)

Tentando enviar ordem: 8. Venda Futuro Ibov | Ativo: WINQ25 | Volume: 1.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132781193 | Preço: 139425.0 | Volume: 1.0


True

In [26]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O contrato futuro vence! O contrato vigente para Mini-Dólar é WDON25 (Julho/25).
contrato_dolar = "WDON25"
numero_contratos = 1.0
# ----------------------------
enviar_ordem(
    symbol=contrato_dolar,
    volume=numero_contratos,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="9. Compra Futuro Dolar"
)

Tentando enviar ordem: 9. Compra Futuro Dolar | Ativo: WDON25 | Volume: 1.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132781540 | Preço: 5505.0 | Volume: 1.0


True

In [27]:
# --- Parâmetros Editáveis ---
# ATENÇÃO: O contrato futuro vence! O contrato vigente para Mini-Dólar é WDON25 (Julho/25).
contrato_dolar = "WDON25"
numero_contratos = 1.0
# ----------------------------
enviar_ordem(
    symbol=contrato_dolar,
    volume=numero_contratos,
    tipo_ordem=mt5.ORDER_TYPE_SELL,
    comentario="10. Venda Futuro Dolar"
)

Tentando enviar ordem: 10. Venda Futuro Dolar | Ativo: WDON25 | Volume: 1.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132781668 | Preço: 5504.0 | Volume: 1.0


True

In [28]:
# --- Parâmetros Editáveis ---
ativo_etf = "XFIX11"
volume_compra = 200.0
# ----------------------------
enviar_ordem(
    symbol=ativo_etf,
    volume=volume_compra,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="11. Compra ETF Imobiliario"
)

Tentando enviar ordem: 11. Compra ETF Imobiliario | Ativo: XFIX11 | Volume: 200.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132781806 | Preço: 12.27 | Volume: 200.0


True

In [29]:
# --- Parâmetros Editáveis ---
ativo_etf = "IVVB11"
volume_compra = 50.0
# ----------------------------
enviar_ordem(
    symbol=ativo_etf,
    volume=volume_compra,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="12. Compra ETF Internacional"
)

Tentando enviar ordem: 12. Compra ETF Internacional | Ativo: IVVB11 | Volume: 50.0
--- ORDEM ENVIADA COM SUCESSO ---
Ticket: 2132782132 | Preço: 378.52 | Volume: 50.0


True

In [9]:
# --- Parâmetros Editáveis ---
ativo_etf = "IMAB11"
volume_compra = 100.0
# ----------------------------
enviar_ordem(
    symbol=ativo_etf,
    volume=volume_compra,
    tipo_ordem=mt5.ORDER_TYPE_BUY,
    comentario="13. Compra ETF Renda Fixa"
)

Tentando enviar ordem: 13. Compra ETF Renda Fixa | Ativo: IMAB11 | Volume: 100.0
O ativo 'IMAB11' não foi encontrado.


False

In [13]:
terminal = mt5.terminal_info()